In [26]:
import numpy as np
import igraph as ig
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
import seaborn as sns
import random
from numpy import linalg

In [31]:
## REACTION MODEL Mimura - Murray ##
a = 35
b = 16
c = 9
d = 2/5
## Diffusion parameters ##
epsilon = 0.12
sigma = 15.6
## Reaction Model related quantities ##
def compute_fixed_point(a = a, b = b, c = c, d = d):
    coeff = [d*d, (c + 2*d - b * d), (1 - a - b)]
    roots = np.roots(coeff)
    v = max(roots)
    u = 1 + d * v
    return [u,v]

def compute_jacobian(a = a, b = b, c = c, d = d):
    [u, v] = compute_fixed_point()
    f_partial_u = (1 / c) * (2 * b * u - 3 * u * u + a - c * v)
    f_partial_v = - u
    g_partial_u = + v
    g_partial_v = - 2 * d * v + u - 1
    J = np.zeros((2, 2))
    J[0, 0] = f_partial_u
    J[0, 1] = f_partial_v
    J[1, 0] = g_partial_u
    J[1, 1] = g_partial_v
    return J

def compute_critical_sigma():
    J = compute_jacobian()
    sigma = (np.linalg.det(J) + np.abs(J[0,1] * J[1, 0]) + 2 * np.sqrt(np.linalg.det(J) * np.abs(J[0,1] * J[1, 0]))) / np.square(J[0, 0])
    return sigma

## Diffusion related quantities ##

def compute_critical_eigenvalue(epsilon = epsilon):
    sigma = compute_critical_sigma()
    J = compute_jacobian()
    num = sigma * (J[0, 0] - J[1, 1]) - (sigma + 1) * np.sqrt(sigma * np.abs(J[1,1]) * J[1, 0])    
    den = epsilon * sigma * (sigma - 1)
    eig = num / den
    return eig

#def compute_upper_branch(epsilon = epsilon):


## NUMERICAL INTEGRATION ##

def vector_field(u, v, a = a, b = b, c = c, d = d):
    f = ((a + b * u - u * u) / c - v) * u
    g = (u - (1 + d * v)) * v
    return [f, g]

def perturbed_vec(u, perturbation_amplitude = 0.1):
    for index in np.arange(0, len(u)):
        u[index] += random.uniform(-perturbation_amplitude, + perturbation_amplitude)
    return u

# Graph functions #

# define a Barabasi-Albert graph with node indices sorted by degree in descending order (hubs first)
def ba_graph(n, mean_degree):
    m = int(mean_degree/2)  
    ba_graph = ig.Graph.Barabasi(n, m)
    degrees = ba_graph.degree()
    sorted_indices = sorted(range(len(degrees)), key=lambda k: degrees[k], reverse=True)
    new_indices = {old_idx: new_idx for new_idx, old_idx in enumerate(sorted_indices)}
    reindexed_edges = [(new_indices[edge.source], new_indices[edge.target]) for edge in ba_graph.es]
    graph = ig.Graph(edges=reindexed_edges)
    return graph

def find_closest_eigenvalue(g, critical_eigenvalue):
    laplacian_matrix = np.array(g.laplacian())
    eigenvalues, eigenvectors = np.linalg.eigh(laplacian_matrix)
    chosen_index = 0
    for index, eig in enumerate(eigenvalues):



In [29]:
## Graph Parameters ##
num_nodes = 1000
mean_degree = 20

u_eq = np.full(num_nodes, compute_fixed_point()[0], dtype= float)
v_eq = np.full(num_nodes, compute_fixed_point()[1],  dtype= float)

u_0 = perturbed_vec(u_eq)
g = ba_graph(num_nodes, mean_degree)
laplacian_matrix = np.array(g.laplacian())
eigenvalues, eigenvectors = np.linalg.eigh(laplacian_matrix)

In [34]:
compute_critical_eigenvalue(epsilon= 0.06)

-22.033138203296854